In [10]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


Read in the web page containing the postal code info

In [11]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')


In [12]:
list(soup.children)
[type(item) for item in list(soup.children)]
table = soup.find_all('table')[0]
num_rows = len(table.findAll('tr'))
df = pd.DataFrame(columns = ['Postcode', 'Borough', 'Neighbourhood'], index = range(num_rows))
df.head()

,Postcode,Borough,Neighbourhood
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [13]:
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker += 1
df

,Postcode,Borough,Neighbourhood
0,NaN,NaN,NaN
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n
6,M5A,Downtown Toronto,Regent Park\n
7,M6A,North York,Lawrence Heights\n
8,M6A,North York,Lawrence Manor\n
9,M7A,Queen's Park,Not assigned\n


Remove rows with Borough == "Not assigned"
If Borough exists but Neighborhood == "Not assigned" set Neighborhood name = Borough name

In [14]:
df.dropna(inplace = True)
df.reset_index(inplace = True, drop = True)

In [15]:
df.drop(df[df['Borough'] == 'Not assigned'].index, axis = 0, inplace = True)
df.reset_index(drop = True, inplace = True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n
5,M6A,North York,Lawrence Manor\n
6,M7A,Queen's Park,Not assigned\n
7,M9A,Etobicoke,Islington Avenue\n
8,M1B,Scarborough,Rouge\n
9,M1B,Scarborough,Malvern\n


In [16]:
df = df.replace('\n', '', regex = True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [17]:
for index, row in df.iterrows():
    if (row['Neighbourhood'] == 'Not assigned'):
        row['Neighbourhood'] = row['Borough']

df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Now bunch up multiple neighborhoods in same borough

In [19]:
unique_codes = set(df.Postcode)
postcodes = sorted(unique_codes)

postalcodes      = []
borough       = []
neighbourhood = []

for pcode in postcodes:
    code_item = ''
    borough_item = ''
    neighbor_item = ''    
        
    for code_idx, code_element in enumerate(df.Postcode):
        if code_element == pcode:
            code_item = code_element
            borough_item =df.Borough[code_idx]
            
            if neighbor_item == '': 
                neighbor_item = df.Neighbourhood[code_idx]
            else:
                neighbor_item = neighbor_item + ', ' + df.Neighbourhood[code_idx]
                
    postalcodes.append(code_item)
    borough.append(borough_item)
    neighbourhood.append(neighbor_item)

Make column dictionaries from the lists and then create the dataframe from the dicts.

In [20]:
toronto_dict = {'PostalCode':postalcodes, 'Borough':borough, 'Neighbourhood':neighbourhood}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


write the df to a csv file for use in next part.

In [21]:
df_toronto.to_csv('toronto_postal_codes.csv')

In [22]:
df_toronto.shape

(103, 3)